with reference from https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-vector-search/eval/ground-truth-data.ipynb

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


In [2]:
documents_raw

[{'course': 'data-engineering-zoomcamp',
  'documents': [{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
    'section': 'General course-related questions',
    'question': 'Course - When will the course start?'},
   {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
    'section': 'General course-related questions',
    'question': 'Course - What are the prerequisites for this course?'},
   {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in

In [3]:

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [5]:
for doc in documents:
    doc['id'] = generate_document_id(doc)
documents[3]


{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [6]:
from collections import defaultdict
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)
len(hashes), len(documents)

(947, 948)

In [7]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))
hashes['593f7569']


593f7569 2


[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [8]:
import json
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)
!head documents-with-ids.json


[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [28]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Output should be in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [29]:
from ai21 import AI21Client
from ai21.models.chat import UserMessage

# One way of passing your key to the client.
import os
AI21_API_KEY = os.environ["AI21_API_KEY"]
client = AI21Client(api_key=AI21_API_KEY)

def single_message_instruct(content):
    messages = [UserMessage(content=content)]
    response = client.chat.completions.create(
        model="jamba-1.5-large",
        messages=messages,
        top_p=1.0 # Setting to 1 encourages different responses each call.
    )
    return response.to_json()


def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    messages = [UserMessage(content=prompt)]
    response = client.chat.completions.create(
        model="jamba-1.5-large",
        messages=messages,
        top_p=1.0 # Setting to 1 encourages different responses each call.
    )

    json_response = response.choices[0].message.content
    return json_response

In [30]:

from tqdm.auto import tqdm
results = {}


In [ ]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|                                                                                                                                                 | 0/948 [00:00<?, ?it/s]

In [15]:

# Open a file and use dump() 
with open('results.bin', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(results, file) 

In [16]:
import pickle
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)
results['1f6520ca']
#'["Where can I find the prerequisites for this course?", 
# "How do I check the prerequisites for this course?", 
# "Where are the course prerequisites listed?",
# "What are the requirements for joining this course?",
# "Where is the list of prerequisites for the course?"]'


'{\n  "questions": [\n    "What are the prerequisites for this course?",\n    "Where can I find the prerequisites for this course?",\n    "Are there any specific skills required before taking this course?",\n    "Do I need to have prior knowledge of data engineering before enrolling?",\n    "What topics should I be familiar with before starting this course?"\n  ]\n}'

In [23]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)["questions"]
doc_index = {d['id']: d for d in documents}

TypeError: list indices must be integers or slices, not str

In [27]:
for doc_id, json_questions in results.items():
    try:
        print(json.loads(json_questions))
    except:
        print(json_questions)
    

{'questions': ['When will the course start?', 'What time does the course begin?', 'How can I register for the course?', 'Is there a Telegram channel for the course?', 'Do I need to join a Slack channel for the course?']}
{'questions': ['What are the prerequisites for this course?', 'Where can I find the prerequisites for this course?', 'Are there any specific skills required before taking this course?', 'Do I need to have prior knowledge of data engineering before enrolling?', 'What topics should I be familiar with before starting this course?']}
['Can I still join the course after the start date?', "What happens if I don't register for the course?", 'Can I submit homeworks without registering?', 'Are there deadlines for turning in final projects?', 'What should I be aware of if I join the course late?']
{'questions': ['When will I receive the confirmation email for the Data Engineering Bootcamp?', 'Do I need to wait for the confirmation email to start the Data Engineering Bootcamp?', 

In [ ]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))


In [ ]:
import pandas as pd
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])
df.to_csv('ground-truth-data.csv', index=False)
!head ground-truth-data.csv